In [8]:
#1
import pandas as pd
import numpy as np
import json

import psycopg2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import xgboost as xgb

from sklearn import metrics
from xgboost import plot_importance
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor   
import seaborn as sns
%matplotlib inline
import pickle
import gc
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
plt.style.use('bmh')
from pandas import Series, DataFrame
from sklearn.model_selection import GridSearchCV

import traceback
import sys

import re
import datetime
datetime.timedelta()


datetime.timedelta(0)

In [9]:
def process_call_duration(row):
    if row['Dialer_Disposition'] in ['Answered', 'Agent_Answered']:
        return row['call_duration']
    else:
        return 0

# Script-1

In [14]:
def overall_comms_features_script_1(allocation,comms,script_4_comms):
    
    comms=comms[comms['Loan_ID'].isin(allocation['AgreementNumber'])]
    comms['Loan_ID']=comms['Loan_ID'].astype(int).astype(str)
    
    script_4_comms['Loan_ID']=script_4_comms['Loan_ID'].astype(int).astype(str)
    
    comms['Loan_ID']=comms['Loan_ID'].astype(int).astype(str)
    comms.shape
    
    script_4_comms_loan_ID=script_4_comms.drop_duplicates(subset ="Loan_ID",
                     keep = 'first', inplace = False)
    script_4_comms_loan_ID.shape
    comms[(comms.Loan_ID.isin(script_4_comms_loan_ID.Loan_ID))].shape
    comms_remain=comms[~(comms.Loan_ID.isin(script_4_comms_loan_ID.Loan_ID))]
    
    comms_remain.shape
    script_4_comms.shape
    comms=pd.concat([script_4_comms,comms_remain])
    comms.shape
    comms['Call_Start_Time'] = pd.to_datetime(comms['Call_Start_Time'], errors='coerce')
    comms['day']=comms['Call_Start_Time'].dt.day
    #comms=Jan22[(comms['day']>=0) & (comms['day']<=31)]
    #comms.shape
    
    comms['Loan_ID']=pd.to_numeric(comms['Loan_ID'], errors='coerce')
    #type(comms['Loan_ID'][0])
    comms.shape
    

    ## fetching number of comms on each loan id
    atleast3_calls=comms.groupby(['Loan_ID']).agg(comms_count=('Contact No ','count')).reset_index()
    atleast3_calls.shape
    
        ## merging number of comms on each loan id
    comms=pd.merge(comms,atleast3_calls,how='left',left_on='Loan_ID',right_on='Loan_ID')

    comms=comms[comms.comms_count>=3]
    comms.shape

    comms['Call_Start_Time'] = pd.to_datetime(comms['Call_Start_Time'], errors='coerce')
    comms['Call_End_Time'] = pd.to_datetime(comms['Call_End_Time'], errors='coerce')
    comms['call_duration'] = (comms.Call_End_Time-comms.Call_Start_Time).dt.total_seconds()

    comms['call_duration'] = comms.apply(lambda x: process_call_duration(x), axis=1)

    comms['call_picked']=comms['call_duration'].apply(lambda x: 1 if x>0 else 0)
    comms['call_responsive']=comms['call_duration'].apply(lambda x: 1 if x>=30 else 0)

    ## Aggregating the data
    comms_result=comms.groupby('Loan_ID').agg(calls_count=('Contact No ','count'),
                                             total_call_duration=('call_duration','sum'),
                                             mean_call_duration=('call_duration','mean'),
                                             median_call_duration=('call_duration','median'),
                                             total_call_picked=('call_picked','sum'),
                                             total_call_responsive=('call_responsive','sum')
                                            # paid_flag=('paid_flag','first')
                                             ).reset_index()

    comms_result['call_picked_%']=comms_result['total_call_picked']/comms_result['calls_count']*100
    comms_result['call_response_%']=comms_result['total_call_responsive']/comms_result['calls_count']*100

    return comms_result

# Script-2

In [16]:
def last_3_comms_features_script_2(allocation,comms,script_4_comms,AGENT_DISP_1_2):
    
    #disposition data
    AGENT_DISP_1_2=list(AGENT_DISP_1_2.AGENT_DISP_1_2)
    
    
    comms=comms[comms['Loan_ID'].isin(allocation['AgreementNumber'])]
    comms['Loan_ID']=comms['Loan_ID'].astype(int).astype(str)
      
    script_4_comms['Loan_ID']=script_4_comms['Loan_ID'].astype(int).astype(str)
    comms['Loan_ID']=comms['Loan_ID'].astype(int).astype(str)
    comms.shape
    
    script_4_comms_loan_ID=script_4_comms.drop_duplicates(subset ="Loan_ID",
                     keep = 'first', inplace = False)

    script_4_comms_loan_ID.shape
    comms[(comms.Loan_ID.isin(script_4_comms_loan_ID.Loan_ID))].shape  
    
    comms_remain=comms[~(comms.Loan_ID.isin(script_4_comms_loan_ID.Loan_ID))]

    comms_remain.shape
    script_4_comms.shape
    comms=pd.concat([script_4_comms,comms_remain])
    comms.shape
    
    comms=comms.rename(columns={'AGENT_DISP':'AGENT_DISP_1','AGENT_Sub_DISP':'AGENT_DISP_2'})
    comms.shape
   

    comms['Loan_ID']=pd.to_numeric(comms['Loan_ID'], errors='coerce')

   
    comms.shape
    atleast3_calls=comms.groupby(['Loan_ID']).agg(comms_count=('Contact No ','count')).reset_index()
    atleast3_calls.shape
    comms_v1=pd.merge(comms,atleast3_calls,how='left',left_on='Loan_ID',right_on='Loan_ID')
    comms_v1=comms_v1[comms_v1.comms_count>=3]
    comms_v1.shape

    comms_v1['Call_Start_Time'] = pd.to_datetime(comms_v1['Call_Start_Time'], errors='coerce')
    comms_v1['Call_End_Time'] = pd.to_datetime(comms_v1['Call_End_Time'], errors='coerce')
    comms_v1['call_duration'] = (comms_v1.Call_End_Time-comms_v1.Call_Start_Time).dt.total_seconds()


    def process_call_duration(row):
        if row['Dialer_Disposition'] in ['Answered', 'Agent_Answered']:
            return row['call_duration']
        else:
            return 0

    comms_v1['call_duration'] = comms_v1.apply(lambda x: process_call_duration(x), axis=1)

    comms_v1['call_picked']=comms_v1['call_duration'].apply(lambda x: 1 if x>0 else 0)
    comms_v1['call_responsive']=comms_v1['call_duration'].apply(lambda x: 1 if x>=30 else 0)


    comms_v1.shape
    comms_v2=comms_v1.sort_values('Call_Start_Time').groupby('Loan_ID').tail(3)
    comms_v2.shape

    comms_v2=comms_v2.sort_values(['Loan_ID','Call_Start_Time'])
    #comms_v2["call_flag"] = comms_v2.groupby("Loan_ID")["Call_Start_Time"].rank(method="first", ascending=False)

    call_flag=[]
    for i in range(0,len(comms_v2),3):
        call_flag.extend([3,2,1])
    comms_v2['call_flag']=call_flag

    L3=comms_v2[comms_v2.call_flag==3]
    L2=comms_v2[comms_v2.call_flag==2]
    L1=comms_v2[comms_v2.call_flag==1]

    L3=L3[['Loan_ID','call_duration','call_picked','call_responsive','AGENT_DISP_1','AGENT_DISP_2']].reset_index()
    ##AGENT_DISP_1
    temp_data = pd.get_dummies(L3['AGENT_DISP_1'])
    L3 = L3.drop(columns=['AGENT_DISP_1'], axis=1).join(temp_data)
    L3.shape

    ##AGENT_DISP_2

    ## add columns from the value counts
    temp_data = pd.get_dummies(L3['AGENT_DISP_2'])
    L3 = L3.drop(columns=['AGENT_DISP_2'], axis=1).join(temp_data)
    L3.shape


    ## L3 = L3.fillna(0)

    COL_L3=list(L3.columns)
    for i in AGENT_DISP_1_2:
        if COL_L3.count(i)==0:
            L3[i]=0

    L3=L3.add_prefix('L3_')

    L2=L2[['Loan_ID','call_duration','call_picked','call_responsive','AGENT_DISP_1','AGENT_DISP_2']].reset_index()
    ##AGENT_DISP_1
    temp_data = pd.get_dummies(L2['AGENT_DISP_1'])
    L2 = L2.drop(columns=['AGENT_DISP_1'], axis=1).join(temp_data)
    L2.shape

    ##AGENT_DISP_2
    temp_data = pd.get_dummies(L2['AGENT_DISP_2'])
    L2 = L2.drop(columns=['AGENT_DISP_2'], axis=1).join(temp_data)
    L2.shape

    COL_L2=list(L2.columns)
    for i in AGENT_DISP_1_2:
        if COL_L2.count(i)==0:
            L2[i]=0

    L2=L2.add_prefix('L2_')


    L1=L1[['Loan_ID','call_duration','call_picked','call_responsive','AGENT_DISP_1','AGENT_DISP_2']].reset_index()
    ##AGENT_DISP_1
    temp_data = pd.get_dummies(L1['AGENT_DISP_1'])
    L1 = L1.drop(columns=['AGENT_DISP_1'], axis=1).join(temp_data)
    L1.shape

    ##AGENT_DISP_2
    temp_data = pd.get_dummies(L1['AGENT_DISP_2'])
    L1 = L1.drop(columns=['AGENT_DISP_2'], axis=1).join(temp_data)
    L1.shape

    COL_L1=list(L1.columns)
    for i in AGENT_DISP_1_2:
        if COL_L1.count(i)==0:
            L1[i]=0

    L1=L1.add_prefix('L1_')

    
    ### Merging data
    df1=pd.merge(L3,L2,how='left',left_on='L3_Loan_ID',right_on='L2_Loan_ID')
    comms_last_3calls=pd.merge(df1,L1,how='left',left_on='L2_Loan_ID',right_on='L1_Loan_ID')
    comms_last_3calls.shape

    return comms_last_3calls


# Script-3 

In [4]:
def loan_data_features_script_3(allocation,comms,script_4_comms,AGENT_DISP_1_2):
    
    #disposition data
    AGENT_DISP_1_2=list(AGENT_DISP_1_2.AGENT_DISP_1_2)
    
 
    comms=comms[comms['Loan_ID'].isin(allocation['AgreementNumber'])]
    comms['Loan_ID']=comms['Loan_ID'].astype(int).astype(str)
      
    script_4_comms['Loan_ID']=script_4_comms['Loan_ID'].astype(int).astype(str)
    comms['Loan_ID']=comms['Loan_ID'].astype(int).astype(str)
    comms.shape
    
    script_4_comms_loan_ID=script_4_comms.drop_duplicates(subset ="Loan_ID",
                     keep = 'first', inplace = False)

    script_4_comms_loan_ID.shape
    comms[(comms.Loan_ID.isin(script_4_comms_loan_ID.Loan_ID))].shape  
    
    comms_remain=comms[~(comms.Loan_ID.isin(script_4_comms_loan_ID.Loan_ID))]

    comms_remain.shape
    script_4_comms.shape
    comms=pd.concat([script_4_comms,comms_remain])
    comms.shape
    
    comms=comms.rename(columns={'AGENT_DISP':'AGENT_DISP_1','AGENT_Sub_DISP':'AGENT_DISP_2'})
    comms.shape

   

#     ##June_month/July/August
#     allocation['total_due/Claim Amount']=allocation['OS']
#     allocation['over_prin']=allocation['overdue_principal']
#     allocation['over_penalty']=allocation['Late Payment Fee']+allocation['Fee']
#     allocation['over_interest']=allocation['overdue_interest']+allocation['initial_interest']
#     allocation['Loan_ID']=allocation['AgreementNumber']
#     allocation['loan_amount']=allocation['InitialAmount']
    
    
    ##May_month
    allocation['total_due/Claim Amount']=allocation['OS']
    allocation['over_prin']=allocation['overdue_principal']
    allocation['over_penalty']=allocation['Late Payment charges']+allocation['Fee']
    allocation['over_interest']=allocation['overdue_interest']+allocation['Interest Charged']
    allocation['Loan_ID']=allocation['AgreementNumber']
    allocation['loan_amount']=allocation['InitialAmount']
    
    allocation=allocation.rename(columns={'loan_id':'Loan_ID','Loan ID':'Loan_ID','mobile':'Contact No ','total_due/Claim Amount':'Claim AMT'})

    allocation.shape
    atleast3_calls=comms.groupby(['Loan_ID']).agg(comms_count=('Contact No ','count')).reset_index()
    atleast3_calls.shape

    allocation['Loan_ID']=allocation['Loan_ID'].astype(str)
    atleast3_calls['Loan_ID']=atleast3_calls['Loan_ID'].astype(str)
    
    atleast3_calls.shape

    allocation_v1=pd.merge(allocation,atleast3_calls,how='left',left_on='Loan_ID',right_on='Loan_ID')
    allocation_v2=allocation_v1[allocation_v1.comms_count>=3]

    allocation_v2.shape
    
    allocation_v2=allocation_v2[['Loan_ID','Claim AMT','over_prin','over_penalty','over_interest','loan_amount']]
    allocation_v2['Principle_due_%']=allocation_v2['over_prin']/allocation_v2['Claim AMT']*100
    allocation_v2['Charges_due_%']=(allocation_v2['over_penalty']+allocation_v2['over_interest'])/allocation_v2['Claim AMT']*100
    allocation_v2['Panelty_due_%']=allocation_v2['over_penalty']/allocation_v2['Claim AMT']*100
    allocation_v2['Principle_paid_%']=allocation_v2['over_prin']/allocation_v2['loan_amount']*100
    
    return allocation_v2


# Script-4

In [2]:
def comms_1_2_picked_customers_script_4(allocation,comms):
    
    comms=comms[comms['Loan_ID'].isin(allocation['AgreementNumber'])]
    
    comms['Loan_ID']=comms['Loan_ID'].astype(int).astype(str)
    comms['Call_Start_Time'] = pd.to_datetime(comms['Call_Start_Time'], errors='coerce')
    comms['Call_End_Time'] = pd.to_datetime(comms['Call_End_Time'], errors='coerce')
    comms['call_duration'] = (comms.Call_End_Time-comms.Call_Start_Time).dt.total_seconds()
    
    comms['call_duration'] = comms.apply(lambda x: process_call_duration(x), axis=1)
    
    comms['call_picked']=comms['call_duration'].apply(lambda x: 1 if x>0 else 0)
    comms['call_responsive']=comms['call_duration'].apply(lambda x: 1 if x>=30 else 0)
    
    comms_v2=comms.groupby('Loan_ID').agg(calls_count=('Contact No ','count'),
                                         total_call_duration=('call_duration','sum'),
                                         mean_call_duration=('call_duration','mean'),
                                         median_call_duration=('call_duration','median'),
                                         total_call_picked=('call_picked','sum'),
                                         total_call_responsive=('call_responsive','sum')
                                        # paid_flag=('paid_flag','first')
                                         ).reset_index()

    comms_v2['call_picked_%']=comms_v2['total_call_picked']/comms_v2['calls_count']*100
    comms_v2['call_response_%']=comms_v2['total_call_responsive']/comms_v2['calls_count']*100
    
    comms_v3=comms_v2[(comms_v2.calls_count==1) & (comms_v2.total_call_picked>=1)]
    
    comms_v4=comms[comms.Loan_ID.isin(comms_v3.Loan_ID)]
    comms_v4.shape
    
    comms_v4.shape
    comms_v4_3 = pd.concat([comms_v4]*3, ignore_index=True)
    comms_v4_3.shape
    
    
    
    comms_v4_3.shape
    comms_v5=comms_v4_3.sort_values('Call_Start_Time').groupby('Loan_ID').tail(3).reset_index()
    comms_v5.shape

    comms_v5=comms_v5.sort_values(['Loan_ID','Call_Start_Time']).reset_index()
    #comms_v2["call_flag"] = comms_v2.groupby("Loan_ID")["Call_Start_Time"].rank(method="first", ascending=False)

    
    call_flag=[]
    for i in range(0,len(comms_v5),3):
        call_flag.extend([1,2,3])
    comms_v5['call_flag']=call_flag

    L1=comms_v5[comms_v5.call_flag==1]
    L2=comms_v5[comms_v5.call_flag==2]
    L3=comms_v5[comms_v5.call_flag==3]


    ##changes on L1
    L1['Call_Start_Time']='2022-08-01 00:00:00'
    L1['Call_End_Time']='2022-08-01 00:00:00'
    L1['Dialer_Disposition']=np.nan
    L1['AGENT_DISP']=np.nan
    L1['AGENT_Sub_DISP']=np.nan
    #changed on L2
    L2['Call_Start_Time']='2022-08-01 00:00:00'
    L2['Call_End_Time']='2022-08-01 00:00:00'
    L2['Dialer_Disposition']=np.nan
    L2['AGENT_DISP']=np.nan
    L2['AGENT_Sub_DISP']=np.nan

    comms_v5=pd.concat([L1,L2,L3])
    comms_v5.shape


    comms_v5['Call_Start_Time'] = pd.to_datetime(comms_v5['Call_Start_Time'], errors='coerce')
    comms_v5['Call_End_Time'] = pd.to_datetime(comms_v5['Call_End_Time'], errors='coerce')
    comms_v5['call_duration'] = (comms_v5.Call_End_Time-comms_v5.Call_Start_Time).dt.total_seconds()

    comms_v5=comms_v5.sort_values(['Loan_ID','Call_Start_Time'],ascending=True)

    comms_1_call=comms_v5.copy()
    
    
    comms_v3=comms_v2[(comms_v2.calls_count==2) & (comms_v2.total_call_picked>=1)]
    
    comms_v3.shape
    
    comms_v4=comms[comms.Loan_ID.isin(comms_v3.Loan_ID)]
    comms_v4.shape
    
    #creation of 1 sample for each loan ID

    comms_v4_temp=comms_v4.drop_duplicates(subset ="Loan_ID",
                         keep = 'first', inplace = False)

    comms_v4_temp.shape
    #comms_v4_3_temp = pd.concat([comms_v4_temp]*2, ignore_index=True)
    comms_v4_3=pd.concat([comms_v4_temp,comms_v4])
    comms_v4_3.shape
    
    
    comms_v4_3.shape
    comms_v5=comms_v4_3.sort_values('Call_Start_Time').groupby('Loan_ID').tail(3).reset_index()
    comms_v5.shape

    comms_v5=comms_v5.sort_values(['Loan_ID','Call_Start_Time']).reset_index()
    #comms_v2["call_flag"] = comms_v2.groupby("Loan_ID")["Call_Start_Time"].rank(method="first", ascending=False)

    call_flag=[]
    for i in range(0,len(comms_v5),3):
        call_flag.extend([1,2,3])
    comms_v5['call_flag']=call_flag

    L1=comms_v5[comms_v5.call_flag==1]
    L2=comms_v5[comms_v5.call_flag==2]
    L3=comms_v5[comms_v5.call_flag==3]


    ##changes on L1
    L1['Call_Start_Time']='2022-08-01 00:00:00'
    L1['Call_End_Time']='2022-08-01 00:00:00'
    L1['Dialer_Disposition']=np.nan
    L1['AGENT_DISP']=np.nan
    L1['AGENT_Sub_DISP']=np.nan

    comms_v5=pd.concat([L1,L2,L3])
    comms_v5.shape


    comms_v5['Call_Start_Time'] = pd.to_datetime(comms_v5['Call_Start_Time'], errors='coerce')
    comms_v5['Call_End_Time'] = pd.to_datetime(comms_v5['Call_End_Time'], errors='coerce')
    comms_v5['call_duration'] = (comms_v5.Call_End_Time-comms_v5.Call_Start_Time).dt.total_seconds()

    comms_v5=comms_v5.sort_values(['Loan_ID','Call_Start_Time'],ascending=True)

    comms_2_call=comms_v5.copy()
    
    comms_1_2_call_comms=pd.concat([comms_1_call,comms_2_call])
    comms_1_2_call_comms.shape
    
    return comms_1_2_call_comms

#### End of Code <-------